In [ ]:
import numpy as np
import pandas as pd
from sklearn.decomposition import IncrementalPCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import polars as pl
import plotly.graph_objects as go

In [ ]:
import dash
from dash import dcc, html

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
file_path = '/Users/zeynali/Desktop/دیتا اسمیران/(experimental) مرداد/mordad.csv'
print("در حال بارگذاری داده‌ها با Polars...")
df = pl.read_csv(file_path)

In [ ]:
print("داده‌ها با موفقیت بارگذاری شدند!")
print("شکل دیتافریم اولیه:", df.shape)

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
time_column = 'datetime'  # نام ستون زمان
feature_columns = [col for col in df.columns if col != time_column]

In [ ]:
df2 = df.select([col for col in feature_columns])

In [ ]:
correlation_matrix_pl = df2.corr()

In [ ]:
title_text = 'Correlation Matrix for Features'
feature_names = correlation_matrix_pl.columns
z_values = correlation_matrix_pl.to_numpy()

fig = go.Figure(data=go.Heatmap(
        z=z_values,
        x=feature_names,
        y=feature_names,
        colorscale='RdBu_r',
        zmin=-1,
        zmax=1,
        colorbar=dict(title="Correlation")))

fig.update_layout(
        title={'text': title_text, 'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
        height=735,
        width= 735,
        xaxis={'tickangle': 45},
        yaxis={'tickangle': 0},)

# fig.write_html("interactive_correlation_heatmap.html")

fig.show()

In [ ]:
print("\n🔍 در حال شناسایی سنسورهای خاموش (با واریانس صفر)...")

# محاسبه انحراف معیار برای هر ستون
stds = df.select(pl.col(feature_columns).std())

# پیدا کردن نام ستون‌هایی که انحراف معیار آن‌ها صفر است
disabled_cols = []
for col in feature_columns:
    if stds.select(pl.col(col)).item() == 0:
        disabled_cols.append(col)

In [ ]:
print(disabled_cols)
print(len(disabled_cols))

In [ ]:
df_disabled = pd.DataFrame(disabled_cols, columns=['ستون‌های حذف شده'])
total_count_html = f"<h3>تعداد کل ستون‌های حذف شده: {len(disabled_cols)}</h3>"

In [ ]:
cols_table_html = df_disabled.to_html(index=False, justify='center')
final_html = total_count_html + cols_table_html

In [ ]:
# file_name = "disabled_columns_report.html"

In [ ]:
# with open(file_name, "w", encoding="utf-8") as f:
#     f.write(final_html)
# print(f"✅ فایل با موفقیت در مسیر: '{file_name}' ذخیره شد.")

In [ ]:
df_clean = df.select([col for col in df.columns if col not in disabled_cols])
print(f"   فیچرهای باقی‌مانده: {len(df_clean.columns)}")

In [ ]:
all_faetures = [col for col in df_clean.columns ]

In [ ]:
print(len(all_faetures))

In [ ]:
print(all_faetures)

In [ ]:
from scipy.stats import skew

In [ ]:
df_clean.head()

In [ ]:
count_columns = [col for col in df_clean.columns if col != time_column]

In [ ]:
mean_values_var = df_clean.select(count_columns).mean().row(0)
std_values_var = df_clean.select(count_columns).std().row(0)

# 5. محاسبه امتیاز Z (Z-Score) و امتیاز ناهنجاری
zscore_expressions = [
    ((pl.col(col) - mean_values_var[i]) / pl.lit(std_values_var[i]))
    for i, col in enumerate(count_columns)
]

df_clean = df_clean.with_columns(
    pl.mean_horizontal(
        [expr**2 for expr in zscore_expressions]
    ).alias("anomaly_score")
)

In [ ]:
threshold = df_clean.select("anomaly_score").to_series().mean() + 3 * df_clean.select("anomaly_score").to_series().std()

In [ ]:
df_anomaly_zscore = df_clean.filter(pl.col("anomaly_score") > threshold)

In [ ]:
print("شکل دیتافریم اولیه:", df_anomaly_zscore.shape)

In [ ]:
df_clean = df_clean.filter(pl.col("anomaly_score") <= threshold)

In [ ]:
print("داده‌ها با موفقیت بارگذاری شدند!")
print("شکل دیتافریم اولیه:", df_clean.shape)

In [ ]:
df_clean = df_clean.drop("anomaly_score")

In [ ]:
df_clean = df_clean.to_pandas()

In [ ]:
print(df_clean.dtypes)

In [ ]:
feature_columns = [col for col in df_clean.columns if col != time_column]

In [ ]:
X = df_clean[feature_columns]

In [ ]:
from sklearn.ensemble import IsolationForest

In [ ]:
model = IsolationForest(n_estimators=300, contamination=0.1, random_state=42)

In [ ]:
df_clean['anomaly_label'] = model.fit_predict(X)

In [ ]:
anomalies_iso_tree = df_clean[df_clean['anomaly_label'] == -1]

In [ ]:
print(f"تعداد کل ردیف‌های دارای ناهنجاری شناسایی شده با Isolation Forest: {len(anomalies_iso_tree)}")
print("\n۵ مورد اول:")
print(anomalies_iso_tree.head())

In [ ]:
df_clean['datetime'] = pd.to_datetime(df_clean['datetime'])

In [ ]:
df_clean.head()

In [ ]:
df_clean = pl.from_pandas(df_clean)

In [ ]:
df_clean.head()

In [ ]:
df_clean = df_clean.filter(pl.col("anomaly_label") == 1)

In [ ]:
df_clean = df_clean.drop("anomaly_label")

In [ ]:
print("داده‌ها با موفقیت بارگذاری شدند!")
print("شکل دیتافریم اولیه:", df_clean.shape)

In [ ]:
print("شکل دیتافریم اولیه:", anomalies_iso_tree.shape)

In [ ]:
anomalies_iso_tree= pl.from_pandas(anomalies_iso_tree)

In [ ]:
print(type(anomalies_iso_tree))
print(type(df_anomaly_zscore))

In [ ]:
anomalies_iso_tree.head()

In [ ]:
anomalies_iso_tree = anomalies_iso_tree.drop("anomaly_label")
df_anomaly_zscore = df_anomaly_zscore.drop("anomaly_score")

In [ ]:
print("شکل دیتافریم اولیه:", df_anomaly_zscore.shape)

In [ ]:
df_concat = pl.concat([anomalies_iso_tree, df_anomaly_zscore], how="vertical")

In [ ]:
print("شکل دیتافریم اولیه:", df_concat.shape)

In [ ]:
df_final = df_concat.unique()

In [ ]:
print("شکل دیتافریم اولیه:", df_final.shape)

In [ ]:
df_final.head()

In [ ]:
df_final = df_final.with_columns(
    pl.col("datetime").str.to_datetime(
        # این فرمت برای تجزیه رشته‌های زمانی با دقت میکروثانیه مناسب است
        format="%Y-%m-%dT%H:%M:%S%.f" 
    )
)

In [ ]:
df_final.head(30)

In [ ]:
print("شکل دیتافریم اولیه:", df_final.shape)

In [ ]:
unique_days = df_final.select(
    pl.col("datetime").dt.date().alias("date")
).unique().sort("date")

In [ ]:
unique_days.head()

In [ ]:
daily_anomaly_counts = (
    df_final.group_by(
        # مرحله ۱: استخراج تاریخ (بدون زمان) و ایجاد ستون 'date'
        pl.col("datetime").dt.date().alias("date")
    )
    .len() # مرحله ۲: شمارش تعداد سطرها در هر گروه (ستون خروجی پیش‌فرض 'len' نام می‌گیرد)
    .sort("len", descending=True) # مرحله ۳: مرتب‌سازی DataFrame بر اساس ستون 'len'
)

In [ ]:
daily_anomaly_counts.head(20)

In [ ]:
daily_anomaly_counts_pd = daily_anomaly_counts.to_pandas()

In [ ]:
pip install jdatetime

In [ ]:
import jdatetime

In [ ]:
def convert_to_shamsi(date_gregorian):
    # اطمینان از اینکه ورودی از نوع datetime.date یا Timestamp باشد
    if isinstance(date_gregorian, pd.Timestamp):
        date_gregorian = date_gregorian.date()
        
    # تبدیل به jdatetime
    date_shamsi = jdatetime.date.fromgregorian(
        year=date_gregorian.year, 
        month=date_gregorian.month, 
        day=date_gregorian.day
    )
    # فرمت دهی به صورت رشته (مثلاً 1403/05/10)
    return date_shamsi.strftime("%Y/%m/%d")

In [ ]:
daily_anomaly_counts_pd['shamsi_date'] = daily_anomaly_counts_pd['date'].apply(convert_to_shamsi)

In [ ]:
daily_anomaly_counts_pd.head(20)

In [ ]:
import plotly.express as px

In [ ]:
fig = px.bar(
    daily_anomaly_counts_pd,
    x="shamsi_date",             # *** استفاده از تاریخ شمسی برای محور X ***
    y="len",
    title="تعداد ناهنجاری‌های روزانه (تاریخ شمسی)",
    labels={
        "shamsi_date": "تاریخ شمسی", 
        "len": "تعداد ناهنجاری"
    },
    template="plotly_white",
)

# تنظیمات برای بهبود نمایش متن فارسی
fig.update_layout(
    font_family="Arial", # تنظیم فونت مناسب برای نمایش فارسی
    title_x=0.5, # قرار دادن عنوان در وسط
)

# تنظیمات محور X برای نمایش بهتر تاریخ‌ها
fig.update_xaxes(
    tickangle=-45, # چرخش تاریخ‌ها برای جلوگیری از تداخل
    dtick=1 # اطمینان از نمایش هر روز (اگر تعداد روزها زیاد است، Plotly خودش تنظیم می‌کند)
)

# ذخیره به صورت فایل HTML
output_html_file = "daily_anomaly_bar_chart_shamsi.html"
# fig.write_html(output_html_file)

print(f"نمودار با تاریخ شمسی در فایل '{output_html_file}' ذخیره شد.")

In [ ]:
hour_of_day = pl.col("datetime").dt.hour()

In [ ]:
df_final_with_segment = df_final.with_columns(
    pl.when(hour_of_day.is_between(7, 13))
        .then(pl.lit("ظهر"))  # 10:00 تا 17:59
    .when(hour_of_day.is_between(14, 19))
        .then(pl.lit("عصر"))  # 18:00 تا 21:59
    .otherwise(pl.lit("شب"))  # 22:00 تا 09:59
    .alias("Time_Segment")
)

# نمایش چند ردیف برای بررسی ستون جدید
print("DataFrame با ستون جدید Time_Segment:")
print(df_final_with_segment[['datetime', 'Time_Segment']].head())

In [ ]:
df_final_with_segment.describe()

In [ ]:
anomaly_counts_by_segment = df_final_with_segment.group_by("Time_Segment").len()

# برای Plotly، باید Polars DataFrame را به Pandas DataFrame تبدیل کنیم.
anomaly_counts_pd = anomaly_counts_by_segment.to_pandas()

print("شمارش ناهنجاری بر اساس بخش زمانی (آماده برای نمودار):")
print(anomaly_counts_pd)

In [ ]:

fig_pie = px.pie(
    anomaly_counts_pd,
    names='Time_Segment',  # ستونی که لیبل‌های برش‌ها را مشخص می‌کند
    values='len',          # ستونی که اندازه برش‌ها را مشخص می‌کند (تعداد ناهنجاری)
    title='توزیع ناهنجاری‌ها بر اساس بخش زمانی (ظهر، عصر، شب)',
    # نمایش درصد روی نمودار
    hole=.3, # ایجاد یک نمودار دایره‌ای با سوراخ (Donut Chart) برای جذابیت بیشتر (اختیاری)
    template="plotly_white"
)

# تنظیمات برای بهبود نمایش فارسی
fig_pie.update_layout(
    font_family="Arial", # تنظیم فونت مناسب برای نمایش فارسی
    title_x=0.5,         # قرار دادن عنوان در وسط
    legend_title="بخش زمانی"
)

# نمایش درصد و مقدار
fig_pie.update_traces(
    textposition='inside', # قرار دادن متن داخل برش‌ها
    textinfo='percent+value' # نمایش درصد و مقدار
)


# --- گام ۳: ذخیره نمودار به عنوان فایل HTML ---
output_html_file = "anomaly_distribution_pie_chart.html"
# fig_pie.write_html(output_html_file)

print(f"نمودار دایره‌ای تعاملی در فایل '{output_html_file}' ذخیره شد.")
fig_pie.show()
fig.show()

In [ ]:
df_final_with_segment.describe()

In [ ]:
df_clean.describe()

In [ ]:
def get_highly_skewed_features_updated(df: pl.DataFrame, threshold: float = 2.0) -> dict:
    """
    محاسبه چولگی و شناسایی فیچرهای با چولگی بالا با استفاده از pl.selectors.numeric()
    """
    
    # گام A: شناسایی ستون‌های عددی با روش جدید (رفع DeprecationWarning)
    # از pl.selectors.numeric() برای انتخاب خودکار ستون‌های عددی استفاده می‌کنیم.
    numeric_cols = df.select(pl.selectors.numeric()).columns
    
    if not numeric_cols:
        return {}
    
    # گام B: ایجاد عبارات برای محاسبه چولگی (بدون تغییر)
    # از pl.selectors.numeric() به طور مستقیم در select نیز می‌توان استفاده کرد.
    
    skew_results_df = df.select(
        [pl.col(col).skew().alias(col) for col in numeric_cols]
    )
    
    # استخراج نتایج چولگی
    skewness_values = skew_results_df.row(0, named=True)
    
    # گام C: فیلتر کردن فیچرهایی که |Skewness| > threshold است
    highly_skewed_features = {
        name: skew
        for name, skew in skewness_values.items()
        if abs(skew) > threshold
    }
    
    return highly_skewed_features

In [ ]:
highly_skewed_map = get_highly_skewed_features_updated(df_clean, threshold=2.0)

In [ ]:
print("### 🔍 نتایج تحلیل چولگی با Polars ###")
print(f"تعداد کل فیچرهای عددی: {len([col for col in df_clean.columns if df_clean[col].dtype in pl.NUMERIC_DTYPES])}")
print(f"تعداد فیچرهای با چولگی بالا (خارج از [-2, 2]): {len(highly_skewed_map)}")

print("\n---")
print("## 🚨 فیچرهای دارای چولگی بالا (Skewness > 2 یا Skewness < -2):")

In [ ]:
if highly_skewed_map:
    # ایجاد DataFrame نهایی مستقیماً از دیکشنری (بهترین روش)
    result_df = pl.DataFrame({
        "Feature": list(highly_skewed_map.keys()),
        "Skewness": list(highly_skewed_map.values())
    }).with_columns(
        # فرمت‌دهی مقدار Skewness برای خوانایی بهتر
        pl.col("Skewness").map_elements(lambda x: f"{x:.3f}", return_dtype=pl.String)
    )
    
    print(result_df)
else:
    print("هیچ فیچری با چولگی بالاتر از 2 یا کمتر از -2 یافت نشد.")

In [ ]:
from typing import Dict


def get_highly_kurtotic_features_updated(df: pl.DataFrame, threshold: float = 2.0) -> Dict[str, float]:
    """
    محاسبه کشیدگی (Kurtosis) و شناسایی فیچرهای با کشیدگی بالا (Leptokurtic)
    با استفاده از pl.selectors.numeric() در Polars.
    
    توجه: کشیدگی Polars به طور پیش‌فرض کشیدگی اضافی (Excess Kurtosis) را محاسبه می‌کند.
    """
    
    # گام A: شناسایی ستون‌های عددی
    # از pl.selectors.numeric() برای انتخاب خودکار ستون‌های عددی استفاده می‌کنیم.
    numeric_cols = df.select(pl.selectors.numeric()).columns
    
    if not numeric_cols:
        print("هشدار: هیچ ستون عددی‌ای برای محاسبه کشیدگی یافت نشد.")
        return {}
    
    # گام B: ایجاد عبارات برای محاسبه کشیدگی (Kurtosis)
    # از تابع .kurtosis() برای محاسبه کشیدگی اضافی استفاده می‌کنیم.
    kurtosis_results_df = df.select(
        [pl.col(col).kurtosis().alias(col) for col in numeric_cols]
    )
    
    # استخراج نتایج کشیدگی
    # row(0, named=True) نتایج را به صورت یک دیکشنری درمی‌آورد.
    kurtosis_values = kurtosis_results_df.row(0, named=True)
    
    # گام C: فیلتر کردن فیچرهایی که |Kurtosis| > threshold است
    # ما معمولاً به دنبال کشیدگی مثبت (Leptokurtic) هستیم که نشان‌دهنده دم‌های سنگین است.
    highly_kurtotic_features = {
        name: kurtosis
        for name, kurtosis in kurtosis_values.items()
        # در اینجا ما هم مقادیر مثبت و هم منفی شدید را فیلتر می‌کنیم
        if abs(kurtosis) > threshold
    }
    
    return highly_kurtotic_features

In [ ]:
kurtosis_threshold = 3.0 
results = get_highly_kurtotic_features_updated(df_clean, threshold=kurtosis_threshold)

print(f"نتیجه کشیدگی برای آستانه |{kurtosis_threshold}|:")
print(results)

In [ ]:
if results:
    # ایجاد DataFrame نهایی مستقیماً از دیکشنری (بهترین روش)
    kurtosis_df = pl.DataFrame({
        "Feature": list(results.keys()),
        "kurtosis": list(results.values())
    })
    
    print(kurtosis_df)
else:
    print("هیچ فیچری با چولگی بالاتر از 2 یا کمتر از -2 یافت نشد.")

In [ ]:
kurtosis_df.describe()

In [ ]:
output_filename = "highly_kurtosis_features.csv"

In [ ]:
kurtosis_df.write_csv(output_filename)

In [ ]:
import polars as pl
import plotly.express as px
import numpy as np
from typing import Optional

# فرض می‌کنیم df دیتافریم Polars شماست و feature_name همان 'f178' است
feature_name = 'f60'
# تبدیل ستون مورد نظر به لیست/آرایه برای Plotly
data_to_plot = df_clean.select(pl.col(feature_name)).to_series().to_list()

def plot_interactive_histogram(data: list, feature: str, title: str):
    """رسم هیستوگرام تعاملی با Plotly."""
    fig = px.histogram(
        x=data,
        nbins=100,  # تعداد سطل‌ها برای وضوح بیشتر
        title=title,
        opacity=0.7,
        template="plotly_white"
    )
    fig.update_layout(xaxis_title="مقدار فیچر", yaxis_title="فراوانی")
    # Show the plot in the notebook/environment
    fig.show()

# 1. رسم هیستوگرام کامل
plot_interactive_histogram(
    data_to_plot, 
    feature_name, 
    f'هیستوگرام کامل تعاملی فیچر: {feature_name}'
)

# 2. رسم هیستوگرام زوم شده (برای تمرکز روی داده‌های اصلی)
# مقادیر بالای 99% را حذف می‌کنیم تا جزئیات مرکز دیده شود
Q99 = np.percentile(data_to_plot, 99)
data_zoomed = [x for x in data_to_plot if x <= Q99]

plot_interactive_histogram(
    data_zoomed, 
    feature_name, 
    f'هیستوگرام تعاملی فیچر: {feature_name} (زوم شده، زیر صدک ۹۹)'
)

In [ ]:

# استفاده از describe() فقط برای ستون مورد نظر
statistical_summary = df_clean.select(pl.col(feature_name)).describe()

print(f"خلاصه‌ی آماری برای فیچر: {feature_name}")
print(statistical_summary)

In [ ]:
print(len(selected_pca))

In [ ]:
skewness_df = df_clean.select([
    pl.col(col).skew().alias(f"skew_{col}")
    for col in selected_pca
]).to_dict(as_series=False)

In [ ]:
skewness_map = {
    col: list(skewness_df[f"skew_{col}"]) [0]
    for col in selected_pca
}

In [ ]:
SKEWNESS_THRESHOLD = 2

In [ ]:
columns_to_drop = [
    col for col, skew_val in skewness_map.items()
    if (skew_val is not None) and (abs(skew_val) > SKEWNESS_THRESHOLD)
]

In [ ]:
print(columns_to_drop)

In [ ]:
df_clean = df_clean.drop(columns_to_drop)

In [ ]:
print("داده‌ها با موفقیت بارگذاری شدند!")
print("شکل دیتافریم اولیه:", df_clean.shape)

In [ ]:
selected_pca = [
    col for col in df_clean.columns
]

In [ ]:
print(len(selected_pca))

In [ ]:
skewness_df = df_clean.select([
    pl.col(col).skew().alias(f"skew_{col}")
    for col in selected_pca
]).to_dict(as_series=False)

In [ ]:
skewness_map = {
    col: list(skewness_df[f"skew_{col}"]) [0]
    for col in selected_pca
}

In [ ]:
import math

In [ ]:
columns_to_drop_nan_inf = []

for col, skew_val in skewness_map.items():
    # بررسی هر سه حالت نامعتبر: None، NaN یا Inf
    # این ساده ترین و امن ترین فیلتر برای مقادیر نامعتبر آماری است.
    is_nan_or_none = skew_val is None or (isinstance(skew_val, float) and math.isnan(skew_val))
    is_inf = isinstance(skew_val, float) and math.isinf(skew_val)

    if is_nan_or_none or is_inf:
        columns_to_drop_nan_inf.append(col)

In [ ]:
print(columns_to_drop_nan_inf)

In [ ]:
df_clean = df_clean.drop(columns_to_drop_nan_inf)

In [ ]:
print("داده‌ها با موفقیت بارگذاری شدند!")
print("شکل دیتافریم اولیه:", df_clean.shape)

In [ ]:
df_clean.head()

In [ ]:
correlation_matrix = df_clean.corr()

In [ ]:
plt.figure(figsize=(30, 30)) # مثال: می توانید این مقادیر را برای دید بهتر تنظیم کنید

# 3. رسم نمودار حرارتی (Heatmap)
sns.heatmap(
    correlation_matrix,
    annot=False,  # به دلیل تعداد زیاد فیچر، نمایش مقادیر (annot=True) باعث شلوغی زیاد می‌شود.
    cmap='coolwarm', # یک color map مناسب برای نمایش همبستگی‌های مثبت و منفی
    fmt=".2f", # فرمت نمایش اعداد (اگر annot=True باشد)
    linewidths=0.5, # خطوط بین سلول‌ها
    linecolor='white',
    cbar=True # نمایش نوار رنگ
)

plt.title('Correlation Matrix for 194 Features', fontsize=30)
plt.show()

In [ ]:
datetime_series = df_clean.select("datetime")

In [ ]:
df_clean = df_clean.drop("datetime")

In [ ]:
X_sample = df_clean.to_numpy()

In [ ]:
print(X_sample)

In [ ]:
from sklearn.preprocessing import RobustScaler

In [ ]:
scaler = RobustScaler()
X_sample_scaled = scaler.fit_transform(X_sample)

In [ ]:
print(f"✅ Shape: {X_sample_scaled.shape}\n")

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
# اجرای PCA
# ابتدا با n_components=None اجرا می‌کنیم تا تمام واریانس را بررسی کنیم.
pca = PCA(n_components=None)
pca.fit(X_sample_scaled)

# محاسبه واریانس تجمعی توضیح داده شده (Cumulative Explained Variance)
explained_variance_ratio_cumulative = np.cumsum(pca.explained_variance_ratio_)

# Plot برای تعیین تعداد مؤلفه‌ها 
# این نمودار به شما کمک می‌کند تا زانویی (Elbow) را پیدا کنید که نشان‌دهنده تعداد بهینه مؤلفه‌هاست.

# تعیین تعداد مؤلفه‌هایی که مثلاً ۹۵٪ واریانس را حفظ می‌کنند.
n_components_90 = np.argmax(explained_variance_ratio_cumulative >= 0.99) + 1
print(f"✅ تعداد مؤلفه‌های لازم برای حفظ 90٪ واریانس: {n_components_90}")

In [ ]:
# ادامه کد Python (فراموش نکنید که کتابخانه matplotlib را import کنید)
import matplotlib.pyplot as plt

# اجرای PCA مجدد با n_components=n_components_95
final_pca = PCA(n_components=2)
final_pca.fit(X_sample_scaled)

print("-" * 40)
print("📌 واریانس توضیح داده شده توسط مؤلفه ها:")
print(f"PC1: {final_pca.explained_variance_ratio_[0]*100:.2f}%")
print(f"PC2: {final_pca.explained_variance_ratio_[1]*100:.2f}%")
print(f"واریانس تجمعی: {final_pca.explained_variance_ratio_.sum()*100:.2f}%")

# رسم نمودار Scree Plot برای مشاهده واضحتر
plt.figure(figsize=(8, 5))
plt.plot(np.cumsum(pca.explained_variance_ratio_), marker='o', linestyle='--')
plt.title('Scree Plot (Cumulative Explained Variance)')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance Ratio')
plt.grid()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
n_components_optimal = 7

In [ ]:
pca_final = PCA(n_components=n_components_optimal)

In [ ]:
X_pca_np = pca_final.fit_transform(X_sample_scaled) # فرض بر این است که X_sample_scaled داده‌های آماده شماست

In [ ]:
print(X_pca_np.shape)

In [ ]:
column_names = [f"PC{i+1}" for i in range(n_components_optimal)]
pca_results_df = pl.DataFrame(
    X_pca_np, 
    schema=column_names
)

In [ ]:
pca_results_df.head(50)

In [ ]:
print(f"شکل (Shape) دیتافریم تبدیل شده: {pca_results_df.shape}")
print(pca_results_df.head())

In [ ]:
final_df = pca_results_df.hstack(datetime_series)